In [3]:
import os
from glob import glob
import numpy as np
import fitsio
from scipy.ndimage import gaussian_filter1d
from matplotlib import pyplot as plt
from time import time
from scipy.interpolate import interp1d
from astropy.io import fits
targetid_abs_g = np.load('/global/u1/z/zhaozhon/task/targetid_abs_g.npy')
targetid_abs_r = np.load('/global/u1/z/zhaozhon/task/targetid_abs_r.npy')
targetid_abs_i = np.load('/global/u1/z/zhaozhon/task/targetid_abs_i.npy')
z_g = np.load('/global/u1/z/zhaozhon/task/z_g.npy')
z_r = np.load('/global/u1/z/zhaozhon/task/z_r.npy')
z_i = np.load('/global/u1/z/zhaozhon/task/z_i.npy')
bp_g = np.load('/global/u1/z/zhaozhon/k-correction/Bandpass/bp_g.npy')
bp_r = np.load('/global/u1/z/zhaozhon/k-correction/Bandpass/bp_r.npy')
bp_i = np.load('/global/u1/z/zhaozhon/k-correction/Bandpass/bp_i.npy')

In [10]:
# Define your TARGETID and output file name here
tid = 39627601458760446  # Replace with your TARGETID
outpng = 'output.png'  # Replace with your desired output file name

# Directory where the FITS files are located
# Replace 'PSCRATCH' with the actual path if the environment variable is not set
ardir = os.path.join(os.getenv("PSCRATCH", "/pscratch/sd/z/zhaozhon"))

def get_ws(ardir):
    fn = sorted(
        glob(os.path.join(ardir, "pernight-spectra","desi-ztf-qso-iron-*-*.fits"))
    )[0]
    ws = fitsio.read(fn, "BRZ_WAVE")
    return ws
def get_tid_tileids_nights(tid, all_tids, all_tileids, all_nights):
    sel = all_tids == tid
    return all_tileids[sel], all_nights[sel]
def get_indiv_spectra(tid, tileids, nights, ardir, nwave):

    nobs = len(tileids)

    # read the spectra
    fs = np.zeros((nobs, nwave))  # flux
    ivs = np.zeros((nobs, nwave))  # inverse variance

    # loop on (tileids, nights)
    for i, (tileid, night) in enumerate(zip(tileids, nights)):

        fn = os.path.join(
            ardir,"pernight-spectra",
            "desi-ztf-qso-iron-{}-{}.fits".format(tileid, night),
        )

        # first get the row corresponding to TARGETID
        # (for a given TILEID, a TARGETID can appear only once max.)
        tmp_fm = fitsio.read(fn, "FIBERMAP", columns=["TARGETID"])
        tmp_i = np.where(tmp_fm["TARGETID"] == tid)[0][0]

        # now read the flux, ivar only for that row

        # this is more intuitive, but it loads the whole image first,
        #   then extracts the relevant row, so it s ~slow
        fs[i, :] = fitsio.read(fn, "BRZ_FLUX")[tmp_i, :]
        ivs[i, :] = fitsio.read(fn, "BRZ_IVAR")[tmp_i, :]

        # this way is *much* faster, with reading only the relevant row
        #   note: fitsio wants/works with a slice, not an integer...
        #h = fitsio.FITS(fn)
        #tmp_slice = slice(tmp_i, tmp_i + 1, 1)
        #fs[i, :] = h["BRZ_FLUX"][tmp_slice, :]
        #ivs[i, :] = h["BRZ_IVAR"][tmp_slice, :]

    return fs, ivs
def get_smooth(fs, ivs, gauss_smooth):
    tmp0fs = fs.copy()
    tmp0fs[ivs == 0] = 0
    tmp1fs = 1 + 0 * fs.copy()
    tmp1fs[ivs == 0] = 0
    tmp0smfs = gaussian_filter1d(tmp0fs, gauss_smooth, mode="constant", cval=0)
    tmp1smfs = gaussian_filter1d(tmp1fs, gauss_smooth, mode="constant", cval=0)
    return tmp0smfs / tmp1smfs
def plot_tid_spectra(outpng, tid, ws, fs, ivs):
    fig, ax = plt.subplots(figsize=(10, 5))
    for i in range(fs.shape[0]):
        smfs_i = get_smooth(fs[i], ivs[i], 5)
        ax.plot(ws, smfs_i, lw=0.5)
    ax.set_title("TARGETID = {}".format(tid))
    ax.set_xlabel("Observed wavelength [A]")
    ax.set_ylabel("Flux [erg / cm2 / s / A]")
    ax.grid()
    plt.savefig(outpng, bbox_inches="tight")
    plt.close()
sumfn = os.path.join(ardir, "desi-ztf-qso-iron-pernight-summary.fits")
def save_wavelength_and_flux_to_npy(wavelength, flux, filename1,filename2):
    np.save(filename1, wavelength)
    np.save(filename2,flux)
def k_correction(ws,fs,ws_bp,bp,z0,z1,x_l,x_r):
    fs_avg = np.mean(fs, axis=0)
    f = interp1d(ws,fs_avg, kind='cubic')
    f_bp = interp1d(ws_bp, bp, kind='cubic')
    #area0
    new_ws = np.linspace(ws_bp[0],ws_bp[-1],10000)
    new_fs = f(new_ws)
    new_bp = f_bp(new_ws)
    y_0 = new_ws*new_fs*new_bp
    area0 = np.trapz(y_0,new_ws)
    #area1
    new_ws1 = np.linspace(ws_bp[0]/(1+z0),ws_bp[-1]/(1+z0),10000)
    new_fs1 = f(new_ws1*(1+z1))
    y_1 = new_ws1*new_fs1*new_bp
    area1 = np.trapz(y_1,new_ws1)
    k = 2.5*np.log10(1+z1)+2.5*np.log10(area1/area0)
    #if 2.1<z1<2.9:       
        #plt.plot(new_ws1,new_fs1)
        #plt.plot(new_ws1,new_bp)
        #plt.show()
    #zero_point
    #area2 = np.trapz(f_z(new_ws)*new_ws*new_bp,new_ws)
    #area3 = np.trapz(f_z(new_ws1*(1+z1))*new_ws1*new_bp,new_ws1)
    #k = 2.5*np.log10(1+z1)+2.5*np.log10(area1/area0)+2.5*np.log10(area2/area3)
    return k
# wavelengths
ws = get_ws(ardir)
nwave = len(ws)

# read the master table
start = time()
d = fitsio.read(sumfn, "FIBERMAP")
print("reading {} done (took {:.1f}s)".format(sumfn, time() - start))

# get the (TILEID,NIGHT) sets for that TARGETID
tileids, nights = get_tid_tileids_nights(tid, d["TARGETID"], d["TILEID"], d["NIGHT"])
print(tileids,nights)

reading /pscratch/sd/z/zhaozhon/desi-ztf-qso-iron-pernight-summary.fits done (took 2.0s)
[1090] [20210517]
